In [2]:
import ee
import geemap
import folium
import time

ee.Initialize()
Map = geemap.Map()

# Define the region of interest of Amazon basin
amazon_region = ee.Geometry.Polygon([[[-80.0, 10.0],
            [-20.0, 10.0],
            [-20.0, -50.0],                           
            [-80.0, -50.0],
            [-80.0, 10.0]]]) 

print("START")

# load sentinel-1 data 
# sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
#              .filterBounds(amazon_region)\
#              .filterDate('2022-01-01', '2022-12-31')\
#              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
#              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
#              .filter(ee.Filter.inList('orbitProperties_pass', ['ASCENDING', 'DESCENDING']));

# print('Number of sentinel1 points:', sentinel1.size().getInfo())


# # Select the VV and VH bands
# sentinel1_vv = sentinel1.select('VV').median()
# sentinel1_vh = sentinel1.select('VH').median()
spring = ee.Filter.date('2021-03-01', '2021-04-20');
lateSpring = ee.Filter.date('2021-04-21', '2021-06-10');
summer = ee.Filter.date('2021-06-11', '2021-08-31');

# Define a masking function
def mask_edges(image):
    edge = image.lt(-30.0)  # Define an edge mask where values are less than -30
    masked_image = image.mask().And(edge.Not())  # Mask out edges
    return image.updateMask(masked_image)  # Apply the mask
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterBounds(amazon_region)\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.inList('orbitProperties_pass', ['ASCENDING', 'DESCENDING']))

# Select the VV and VH bands
sentinel1_vv = sentinel1.select('VV')
sentinel1_vh = sentinel1.select('VH')

# Apply the masking function to each image in the collection
sentinel1_vv_masked = sentinel1_vv.map(mask_edges)
sentinel1_vv_final = ee.Image.cat(
        sentinel1_vv_masked.filter(spring).mean(),
        sentinel1_vv_masked.filter(lateSpring).mean(),
        sentinel1_vv_masked.filter(summer).mean());

# Apply the masking function to each image in the collection
sentinel1_vh_masked = sentinel1_vh.map(mask_edges)
sentinel1_vh_final = ee.Image.cat(
        sentinel1_vh_masked.filter(spring).mean(),
        sentinel1_vh_masked.filter(lateSpring).mean(),
        sentinel1_vh_masked.filter(summer).mean());

Map.addLayer(sentinel1_vh_final, 
  {'min': -25, 'max': 5}, 
  'sentinel1_vh_final'
);
Map.addLayer(sentinel1_vv_final, 
  {'min': -25, 'max': 5}, 
  'sentinel1_vv_final'
);
Map.centerObject(amazon_region, 6)
Map

START


Map(center=[-19.307052700621895, -49.99999999999999], controls=(WidgetControl(options=['position', 'transparen…